In [1]:
# Test reconstruction performance

In [2]:
import os
import random
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

from keras.optimizers import Adam
from models import AE, USAD
from utils import *

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
seed_everything(42)

# AutoEncoder(AE)
- LSTM-AE
- GRU-AE
- TCN-AE

In [3]:
# make dataset
df = pd.read_csv('./datasets/JNU_elec.csv').set_index('timestamp')
seq_data = create_seq(df, seq_len=24)

# split(train | val | test)
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'Train: {train.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (26280, 24, 1)
Val: (8760, 24, 1)
Test: (3601, 24, 1)


In [4]:
# Min-Max norm
min_val, max_val = np.min(train), np.max(train)
train_norm = minmax(train, min_val, max_val)
val_norm = minmax(val, min_val, max_val)
test_norm = minmax(test, min_val, max_val)

# make tf dataset
train_dataset = make_dataset(train_norm, train_norm)
val_dataset = make_dataset(val_norm, val_norm)
test_dataset = make_dataset(test_norm, test_norm)

In [5]:
# LSTM-AE
lstmae = AE(mode='LSTM')
# GRU-AE
gruae = AE(mode='GRU')
# TCN-AE
tcnae = AE(z_dim=32, hidden_dim=32, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2])
lstmae.build(input_shape)
gruae.build(input_shape)
tcnae.build(input_shape)

lstmae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
gruae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
tcnae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')

lstm_time = TimeHistory()
gru_time = TimeHistory()
tcn_time = TimeHistory()

In [6]:
print(lstmae.summary(), gruae.summary(), tcnae.summary())

Model: "ae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  16896     
                                                                 
 decoder (Decoder)           multiple                  115457    
                                                                 
Total params: 132,353
Trainable params: 132,353
Non-trainable params: 0
_________________________________________________________________
Model: "ae_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  12864     
                                                                 
 decoder_1 (Decoder)         multiple                  91137     
                                                                 
Total params: 104,001
Trainable params: 104,

In [7]:
epochs = 5
batch_size = 64

lstmae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[lstm_time])
print(f'LSTM-AE train time: {sum(lstm_time.times):.4f}')

Epoch 1/5
411/411 [==============================] - 8s 10ms/step - loss: 0.0107 - mae: 0.0703 - val_loss: 0.0054 - val_mae: 0.0499
Epoch 2/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0037 - mae: 0.0419 - val_loss: 0.0047 - val_mae: 0.0451
Epoch 3/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0029 - mae: 0.0364 - val_loss: 0.0043 - val_mae: 0.0420
Epoch 4/5
411/411 [==============================] - 4s 9ms/step - loss: 0.0025 - mae: 0.0339 - val_loss: 0.0039 - val_mae: 0.0408
Epoch 5/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0023 - mae: 0.0323 - val_loss: 0.0038 - val_mae: 0.0405
LSTM-AE train time: 23.6793


In [8]:
gruae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[gru_time])
print(f'GRU-AE train time: {sum(gru_time.times):.4f}')

Epoch 1/5


411/411 [==============================] - 7s 11ms/step - loss: 0.0226 - mae: 0.1133 - val_loss: 0.0170 - val_mae: 0.0956
Epoch 2/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0057 - mae: 0.0514 - val_loss: 0.0051 - val_mae: 0.0481
Epoch 3/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0040 - mae: 0.0427 - val_loss: 0.0051 - val_mae: 0.0473
Epoch 4/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0034 - mae: 0.0392 - val_loss: 0.0039 - val_mae: 0.0407
Epoch 5/5
411/411 [==============================] - 4s 10ms/step - loss: 0.0026 - mae: 0.0337 - val_loss: 0.0037 - val_mae: 0.0403
GRU-AE train time: 22.4303


In [9]:
tcnae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[tcn_time])
print(f'TCN-AE train time: {sum(tcn_time.times):.4f}')

Epoch 1/5
411/411 [==============================] - 6s 10ms/step - loss: 0.0084 - mae: 0.0592 - val_loss: 0.0027 - val_mae: 0.0341
Epoch 2/5
411/411 [==============================] - 4s 9ms/step - loss: 0.0018 - mae: 0.0280 - val_loss: 0.0016 - val_mae: 0.0263
Epoch 3/5
411/411 [==============================] - 4s 9ms/step - loss: 0.0011 - mae: 0.0228 - val_loss: 0.0013 - val_mae: 0.0234
Epoch 4/5
411/411 [==============================] - 4s 9ms/step - loss: 7.4110e-04 - mae: 0.0183 - val_loss: 0.0011 - val_mae: 0.0214
Epoch 5/5
411/411 [==============================] - 4s 9ms/step - loss: 5.5771e-04 - mae: 0.0160 - val_loss: 7.5325e-04 - val_mae: 0.0179
TCN-AE train time: 20.5263


# UnSupervisedAnomalyDetection(USAD)
- USAD
- TCN-USAD

In [10]:
# USAD
usad = USAD(hidden_dims=[64, 128], mode='USAD')
# TCN-USAD
tcnusad = USAD(z_dim=32, hidden_dims=32, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2])
usad.build(input_shape)
tcnusad.build(input_shape)

usad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)
tcnusad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)

usad_time = TimeHistory()
tcnusad_time = TimeHistory()

In [11]:
print(usad.summary(), tcnusad.summary())

Model: "usad"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  16704     
                                                                 
 decoder_3 (Decoder)         multiple                  16641     
                                                                 
 decoder_4 (Decoder)         multiple                  16641     
                                                                 
Total params: 49,990
Trainable params: 49,986
Non-trainable params: 4
_________________________________________________________________
Model: "usad_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_4 (Encoder)         multiple                  35104     
                                                                 
 decoder_5 (Decoder)         multiple     

In [12]:
usad_times, _ = usad.fit(train_dataset, val_dataset, epochs)
print(f'USAD train time: {sum(usad_times):.4f}')

Epoch 1/5, G Loss: 0.0153, D Loss: 0.0156 | Time: 19.75 sec
Epoch 1/5, val_G Loss: 0.0001, val_D Loss: 0.0001
Validation Step 136, Validation Generator Loss: 3.30988154928491e-06, Validation Discriminator Loss: 3.7976985822751885e-06
Epoch 2/5, G Loss: 0.0042, D Loss: 0.0042 | Time: 19.55 sec
Epoch 2/5, val_G Loss: 0.0000, val_D Loss: -0.0000
Validation Step 136, Validation Generator Loss: 1.0932079703707132e-06, Validation Discriminator Loss: -5.405212277764804e-08
Epoch 3/5, G Loss: 0.0041, D Loss: 0.0017 | Time: 19.68 sec
Epoch 3/5, val_G Loss: 0.0562, val_D Loss: -0.0281
Validation Step 136, Validation Generator Loss: 0.06971906125545502, Validation Discriminator Loss: -0.03485410287976265
Epoch 4/5, G Loss: 0.0166, D Loss: -0.0070 | Time: 19.94 sec
Epoch 4/5, val_G Loss: 0.0632, val_D Loss: -0.0421
Validation Step 136, Validation Generator Loss: 0.0784219279885292, Validation Discriminator Loss: -0.05228114873170853
Epoch 5/5, G Loss: 0.0270, D Loss: -0.0154 | Time: 20.00 sec
Epoc

In [13]:
tcnusad_times, _ = tcnusad.fit(train_dataset, val_dataset, epochs)
print(f'TCN-USAD train time: {sum(tcnusad_times):.4f}')

Epoch 1/5, G Loss: 0.0209, D Loss: 0.0198 | Time: 48.89 sec
Epoch 1/5, val_G Loss: 0.0055, val_D Loss: 0.0052
Validation Step 136, Validation Generator Loss: 0.0008315368322655559, Validation Discriminator Loss: 0.00084017711924389
Epoch 2/5, G Loss: 0.0150, D Loss: 0.0066 | Time: 48.37 sec
Epoch 2/5, val_G Loss: 0.0419, val_D Loss: -0.0046
Validation Step 136, Validation Generator Loss: 0.047464340925216675, Validation Discriminator Loss: -0.021554986014962196
Epoch 3/5, G Loss: 0.0316, D Loss: -0.0038 | Time: 49.11 sec
Epoch 3/5, val_G Loss: 0.0584, val_D Loss: -0.0281
Validation Step 136, Validation Generator Loss: 0.0703124925494194, Validation Discriminator Loss: -0.034859176725149155
Epoch 4/5, G Loss: 0.0395, D Loss: -0.0124 | Time: 49.67 sec
Epoch 4/5, val_G Loss: 0.0646, val_D Loss: -0.0421
Validation Step 136, Validation Generator Loss: 0.07863763719797134, Validation Discriminator Loss: -0.05228113383054733
Epoch 5/5, G Loss: 0.0450, D Loss: -0.0196 | Time: 49.94 sec
Epoch 5

In [ ]:
# To-Do
# 00. make TCN-USAD(Done)
# 00. train USAD with keras fit
# 01. prepare dataset(LEAD 1.0, EPIC, aihub)(ing)
# 02. test and make a Smart Media Journal Paper

# # 모델 가중치 학습 가능 여부 확인 코드
# for weight in autoencoder.encoder.trainable_weights:
#     print(f"Weight name: {weight.name}, Trainable: {weight.trainable}")
# for weight in autoencoder.decoder.trainable_weights:
#     print(f"Weight name: {weight.name}, Trainable: {weight.trainable}")
# autoencoder.encoder.trainable_weights
# autoencoder.decoder.trainable_weights

In [14]:
# # save model weights
# path = './save/JNU/'
# check_path(path)
# lstmae.save(e_path=path+'LSTM_Encoder.h5', d_path=path+'LSTM_Decoder.h5')
# gruae.save(e_path=path+'GRU_Encoder.h5', d_path=path+'GRU_Decoder.h5')
# tcnae_s.save(e_path=path+'TCN_Encoder_s.h5', d_path=path+'TCN_Decoder_s.h5')
# tcnae_l.save(e_path=path+'TCN_Encoder_l.h5', d_path=path+'TCN_Decoder_l.h5')

In [15]:
# path = './save/JNU/'
# lstmae.load(e_path=path+'LSTM_Encoder.h5', d_path=path+'LSTM_Decoder.h5')